In [2]:
from lightgbm.sklearn import LGBMRanker
import lightgbm as lgb
from datetime import timedelta
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, date, timedelta

In [12]:
INPUT_DIR = '../input/'
OUTPUT_DIR = '../output/'

transactions = pd.read_parquet(os.path.join(OUTPUT_DIR, 'transactions_train.parquet'))
use_cols = ['week', 'customer_id', 'article_id', 'price'] # price迷うね
transactions = transactions[use_cols]

# Price

In [13]:
agg_dict = {
                'price': [ 'mean', 'max', 'count']
            }

agg_df = transactions.groupby(['customer_id', 'week'])[['price']].agg(agg_dict)
agg_df = agg_df.droplevel(0, axis=1)
agg_cols = ['agg_price_' + col for col in agg_df.columns]
agg_df.columns = agg_cols
agg_df.reset_index(drop=False, inplace=True)
agg_df.sort_values(['customer_id', 'week'], inplace=True)
agg_df[agg_cols] = agg_df.groupby(['customer_id'])[agg_cols].cumsum()

test_week_agg_df = agg_df.groupby(['customer_id']).tail(1).copy()
test_week_agg_df.week = 105
agg_df = pd.concat([agg_df, test_week_agg_df])

agg_df['week_diff'] = None
agg_df[['agg_price_mean', 'agg_price_max', 'agg_price_count', 'week_diff']] = agg_df.groupby(['customer_id'])[['agg_price_mean', 'agg_price_max', 'agg_price_count', 'week']].shift()
agg_df['week_diff'] = agg_df['week'] - agg_df['week_diff']
agg_df = agg_df[~agg_df.agg_price_mean.isnull()]
agg_df.head()

,customer_id,week,agg_price_mean,agg_price_max,agg_price_count,week_diff
2,23962613628581,45,0.035309,0.067780,19.0,2.0
5,28847241659200,2,0.026133,0.042356,7.0,1.0
6,28847241659200,4,0.068489,0.084712,8.0,2.0
7,28847241659200,6,0.098981,0.115203,9.0,2.0
8,28847241659200,8,0.126930,0.149085,11.0,2.0


In [14]:
agg_df.to_parquet(os.path.join(OUTPUT_DIR, 'user_price_agg.parquet'), index=False)
del agg_df

# Article_id

In [15]:
agg_df = transactions.groupby(['customer_id', 'article_id', 'week'])[['article_id']].count()
agg_df.columns = ['count']
agg_df = agg_df.reset_index(drop=False)
agg_df = agg_df.sort_values(['customer_id', 'article_id', 'week'])
agg_df['user_article_count'] = agg_df.groupby(['customer_id', 'article_id'])[['count']].cumsum()
agg_df.drop(['count'], axis=1, inplace=True)

test_week_agg_df = agg_df.groupby(['customer_id']).tail(1).copy()
test_week_agg_df.week = 105
agg_df = pd.concat([agg_df, test_week_agg_df])

agg_df[['user_article_count']] = agg_df.groupby(['customer_id', 'article_id'])[['user_article_count']].shift()
agg_df = agg_df[~agg_df.user_article_count.isnull()]

agg_df.head()

,customer_id,article_id,week,user_article_count
12,23962613628581,732842001,45,1.0
66,28847241659200,762846001,95,1.0
90,28847241659200,859076001,92,1.0
168,77117344919861,539723003,7,1.0
170,77117344919861,539723042,6,1.0


In [16]:
agg_df.to_parquet(os.path.join(OUTPUT_DIR, 'user_article_counts.parquet'), index=False)
del agg_df

# Index_group